# Exercice 04 - Lecture et ecriture de fichiers avec Spark

## Objectifs
- Lire des fichiers CSV, JSON et Parquet avec Spark
- Ecrire des donnees dans differents formats
- Comprendre les avantages du format Parquet
- Manipuler des fichiers locaux

---

## 1. Les formats de fichiers en Big Data

```
+------------------------------------------------------------------+
|                    FORMATS DE FICHIERS                           |
+------------------------------------------------------------------+
|                                                                  |
|  CSV (Comma Separated Values)                                    |
|  +----------------------------------------------------------+    |
|  | nom,age,ville                                            |    |
|  | Alice,30,Paris                                           |    |
|  | Bob,25,Lyon                                              |    |
|  +----------------------------------------------------------+    |
|  + Simple, lisible                                               |
|  - Pas de typage, lent a lire                                    |
|                                                                  |
|  JSON (JavaScript Object Notation)                               |
|  +----------------------------------------------------------+    |
|  | {"nom": "Alice", "age": 30, "ville": "Paris"}            |    |
|  | {"nom": "Bob", "age": 25, "ville": "Lyon"}               |    |
|  +----------------------------------------------------------+    |
|  + Flexible, structure imbriquee                                 |
|  - Verbose, pas optimal pour gros volumes                        |
|                                                                  |
|  PARQUET (format colonne)                                        |
|  +----------------------------------------------------------+    |
|  | [binaire compresse avec schema integre]                  |    |
|  +----------------------------------------------------------+    |
|  + Tres rapide, compresse, typage fort                           |
|  + Standard du Big Data                                          |
|  - Non lisible par un humain                                     |
|                                                                  |
+------------------------------------------------------------------+
```

## 2. Initialisation de Spark

In [1]:
from pyspark.sql import SparkSession

# Creer la SparkSession
spark = SparkSession.builder \
    .appName("Lecture Ecriture Fichiers") \
    .getOrCreate()

print("Spark pret !")

Spark pret !


## 3. Creer des donnees de test

In [2]:
# Creer un DataFrame de test
donnees = [
    (1, "Alice", 30, "Paris", 55000.0),
    (2, "Bob", 25, "Lyon", 42000.0),
    (3, "Charlie", 35, "Marseille", 65000.0),
    (4, "Diana", 28, "Bordeaux", 48000.0),
    (5, "Eve", 32, "Toulouse", 52000.0)
]

colonnes = ["id", "nom", "age", "ville", "salaire"]

df = spark.createDataFrame(donnees, colonnes)
df.show()

+---+-------+---+---------+-------+
| id|    nom|age|    ville|salaire|
+---+-------+---+---------+-------+
|  1|  Alice| 30|    Paris|55000.0|
|  2|    Bob| 25|     Lyon|42000.0|
|  3|Charlie| 35|Marseille|65000.0|
|  4|  Diana| 28| Bordeaux|48000.0|
|  5|    Eve| 32| Toulouse|52000.0|
+---+-------+---+---------+-------+



## 4. Ecriture en CSV

In [3]:
# Ecrire en CSV
# - mode("overwrite") : ecrase si le fichier existe
# - header=True : inclut les noms de colonnes

chemin_csv = "/home/jovyan/work/exercices/data/employes_csv"

df.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(chemin_csv)

print(f"Fichier CSV ecrit dans : {chemin_csv}")

Fichier CSV ecrit dans : /home/jovyan/work/exercices/data/employes_csv


In [4]:
# Verifier les fichiers crees
import os

for fichier in os.listdir(chemin_csv):
    print(fichier)

.part-00000-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
.part-00002-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
.part-00004-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
.part-00007-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
.part-00009-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
.part-00011-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv.crc
._SUCCESS.crc
part-00000-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
part-00002-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
part-00004-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
part-00007-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
part-00009-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
part-00011-d88cd15d-249f-4077-adbd-7083d8e7dbd8-c000.csv
_SUCCESS


### Note importante

Spark cree un **dossier** contenant plusieurs fichiers :
- `part-00000-xxx.csv` : les donnees (peut y en avoir plusieurs)
- `_SUCCESS` : indique que l'ecriture s'est bien passee

C'est normal ! Spark est concu pour le traitement distribue.

## 5. Lecture du CSV

In [5]:
# Lire le fichier CSV
df_csv = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(chemin_csv)

df_csv.show()
df_csv.printSchema()

+---+-------+---+---------+-------+
| id|    nom|age|    ville|salaire|
+---+-------+---+---------+-------+
|  3|Charlie| 35|Marseille|65000.0|
|  4|  Diana| 28| Bordeaux|48000.0|
|  5|    Eve| 32| Toulouse|52000.0|
|  1|  Alice| 30|    Paris|55000.0|
|  2|    Bob| 25|     Lyon|42000.0|
+---+-------+---+---------+-------+

root
 |-- id: integer (nullable = true)
 |-- nom: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- ville: string (nullable = true)
 |-- salaire: double (nullable = true)



## 6. Ecriture et lecture en JSON

In [6]:
# Ecrire en JSON
chemin_json = "/home/jovyan/work/exercices/data/employes_json"

df.write \
    .mode("overwrite") \
    .json(chemin_json)

print(f"Fichier JSON ecrit dans : {chemin_json}")

Fichier JSON ecrit dans : /home/jovyan/work/exercices/data/employes_json


In [7]:
# Lire le JSON
df_json = spark.read.json(chemin_json)
df_json.show()

+---+---+-------+-------+---------+
|age| id|    nom|salaire|    ville|
+---+---+-------+-------+---------+
| 35|  3|Charlie|65000.0|Marseille|
| 28|  4|  Diana|48000.0| Bordeaux|
| 32|  5|    Eve|52000.0| Toulouse|
| 30|  1|  Alice|55000.0|    Paris|
| 25|  2|    Bob|42000.0|     Lyon|
+---+---+-------+-------+---------+



## 7. Ecriture et lecture en Parquet (recommande)

In [8]:
# Ecrire en Parquet
chemin_parquet = "/home/jovyan/work/exercices/data/employes_parquet"

df.write \
    .mode("overwrite") \
    .parquet(chemin_parquet)

print(f"Fichier Parquet ecrit dans : {chemin_parquet}")

Fichier Parquet ecrit dans : /home/jovyan/work/exercices/data/employes_parquet


In [9]:
# Lire le Parquet
df_parquet = spark.read.parquet(chemin_parquet)
df_parquet.show()
df_parquet.printSchema()

+---+-------+---+---------+-------+
| id|    nom|age|    ville|salaire|
+---+-------+---+---------+-------+
|  3|Charlie| 35|Marseille|65000.0|
|  4|  Diana| 28| Bordeaux|48000.0|
|  5|    Eve| 32| Toulouse|52000.0|
|  1|  Alice| 30|    Paris|55000.0|
|  2|    Bob| 25|     Lyon|42000.0|
+---+-------+---+---------+-------+

root
 |-- id: long (nullable = true)
 |-- nom: string (nullable = true)
 |-- age: long (nullable = true)
 |-- ville: string (nullable = true)
 |-- salaire: double (nullable = true)



### Avantages de Parquet

```
+------------------------------------------------------------------+
|              POURQUOI PARQUET EST LE STANDARD                    |
+------------------------------------------------------------------+
|                                                                  |
|  1. COMPRESSION                                                  |
|     Fichiers 10x plus petits que CSV                             |
|                                                                  |
|  2. SCHEMA INTEGRE                                               |
|     Pas besoin de deviner les types                              |
|                                                                  |
|  3. LECTURE SELECTIVE                                            |
|     Peut lire seulement certaines colonnes                       |
|                                                                  |
|  4. PERFORMANT                                                   |
|     Optimise pour le traitement analytique                       |
|                                                                  |
+------------------------------------------------------------------+
```

## 8. Comparaison des tailles de fichiers

In [10]:
import os

def taille_dossier(chemin):
    """Calcule la taille totale d'un dossier en octets"""
    total = 0
    for fichier in os.listdir(chemin):
        chemin_fichier = os.path.join(chemin, fichier)
        if os.path.isfile(chemin_fichier):
            total += os.path.getsize(chemin_fichier)
    return total

taille_csv = taille_dossier(chemin_csv)
taille_json = taille_dossier(chemin_json)
taille_parquet = taille_dossier(chemin_parquet)

print(f"Taille CSV     : {taille_csv:,} octets")
print(f"Taille JSON    : {taille_json:,} octets")
print(f"Taille Parquet : {taille_parquet:,} octets")

Taille CSV     : 362 octets
Taille JSON    : 413 octets
Taille Parquet : 8,563 octets


---

## Exercice

**Objectif** : Pratiquer la lecture et l'ecriture de fichiers

**Consigne** :
1. Creez un DataFrame avec des donnees de commandes (id, produit, quantite, prix_unitaire)
2. Ecrivez-le en format CSV dans `data/commandes_csv`
3. Ecrivez-le en format Parquet dans `data/commandes_parquet`
4. Relisez les deux fichiers et affichez-les
5. Comparez les tailles

A vous de jouer :

In [11]:
# Donnees de commandes
commandes = [
    (1, "Laptop", 2, 999.99),
    (2, "Souris", 5, 29.99),
    (3, "Clavier", 3, 79.99),
    (4, "Ecran", 1, 299.99),
    (5, "Casque", 4, 149.99)
]

colonnes = ["id", "produit", "quantite", "prix_unitaire"]

# TODO: Creez le DataFrame
df_commandes = spark.createDataFrame(commandes, colonnes)

# TODO: Ecrivez en CSV
chemin_csv_exo = "data/commandes_csv"

df_commandes.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv(chemin_csv_exo)

# TODO: Ecrivez en Parquet
chemin_parquet_exo = "data/commandes_parquet"

df_commandes.write \
    .mode("overwrite") \
    .parquet(chemin_parquet_exo)

In [12]:
# TODO: Relisez et affichez les fichiers
df_commandes_csv = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv(chemin_csv_exo)
df_commandes_csv.show()

df_commandes_parquet = spark.read.parquet(chemin_parquet_exo)
df_commandes_parquet.show()

+---+-------+--------+-------------+
| id|produit|quantite|prix_unitaire|
+---+-------+--------+-------------+
|  1| Laptop|       2|       999.99|
|  3|Clavier|       3|        79.99|
|  5| Casque|       4|       149.99|
|  2| Souris|       5|        29.99|
|  4|  Ecran|       1|       299.99|
+---+-------+--------+-------------+

+---+-------+--------+-------------+
| id|produit|quantite|prix_unitaire|
+---+-------+--------+-------------+
|  3|Clavier|       3|        79.99|
|  1| Laptop|       2|       999.99|
|  2| Souris|       5|        29.99|
|  5| Casque|       4|       149.99|
|  4|  Ecran|       1|       299.99|
+---+-------+--------+-------------+



---

## Resume

Dans ce notebook, vous avez appris :
- Les 3 formats principaux : **CSV**, **JSON**, **Parquet**
- Comment **ecrire** des donnees avec `df.write`
- Comment **lire** des donnees avec `spark.read`
- **Parquet** est le format recommande pour le Big Data

### Prochaine etape
Dans le prochain notebook, nous apprendrons a stocker nos fichiers dans **MinIO** (notre Data Lake).